In [3]:
import numpy as np; import pandas as pd

# Chapter 12 Advanced pandas

## 12.1 Categorical Data

### Background and Motivation

In [4]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)

In [5]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [8]:
values.unique()

array(['apple', 'orange'], dtype=object)

In [9]:
values.value_counts()

apple     6
orange    2
dtype: int64

In [11]:
# Dimension table
values = pd.Series([0,1,0,0] * 2)

In [12]:
dim = pd.Series(['apple', 'orange'])

In [13]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical Type in pandas

In [14]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [15]:
N = len(fruits)

In [16]:
df = pd.DataFrame({'fruit': fruits,
....: 'basket_id': np.arange(N),
....: 'count': np.random.randint(3, 15, size=N),
....: 'weight': np.random.uniform(0, 4, size=N)},
....: columns=['basket_id', 'fruit', 'count', 'weight'])

In [17]:
df

,basket_id,fruit,count,weight
0,0,apple,11,2.873304
1,1,orange,9,0.954933
2,2,apple,8,1.550908
3,3,apple,9,0.343144
4,4,apple,11,2.529958
5,5,orange,14,3.708602
6,6,apple,6,0.714541
7,7,apple,11,1.842154


In [18]:
fruit_cat = df['fruit'].astype('category')

In [19]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [20]:
c = fruit_cat.values

In [21]:
type(c)

pandas.core.arrays.categorical.Categorical

In [22]:
c

[apple, orange, apple, apple, apple, orange, apple, apple]
Categories (2, object): [apple, orange]

In [23]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [25]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [26]:
# convert a DataFrame column to categorical by assigning the converted result:
df['fruit'] = df['fruit'].astype('category')

In [27]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [28]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

In [29]:
categories = ['foo', 'bar', 'baz']

In [30]:
codes = [0,1,2,0,0,1]

In [31]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)

In [32]:
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

### Computation with Categories

In [34]:
np.random.seed(111)

In [35]:
draws = np.random.randn(1000)

In [36]:
bins = pd.qcut(draws, 4)

In [37]:
bins

[(-3.1559999999999997, -0.665], (-0.0445, 0.618], (0.618, 3.266], (-0.665, -0.0445], (-3.1559999999999997, -0.665], ..., (-0.665, -0.0445], (-0.0445, 0.618], (0.618, 3.266], (0.618, 3.266], (-0.665, -0.0445]]
Length: 1000
Categories (4, interval[float64]): [(-3.1559999999999997, -0.665] < (-0.665, -0.0445] < (-0.0445, 0.618] < (0.618, 3.266]]

In [38]:
bins = pd.qcut(draws, 4, labels = ['Q1', 'Q2', 'Q3', 'Q4'])

In [39]:
bins

[Q1, Q3, Q4, Q2, Q1, ..., Q2, Q3, Q4, Q4, Q2]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [40]:
bins.codes[:5]

array([0, 2, 3, 1, 0], dtype=int8)

In [42]:
bins_1 = pd.Series(bins, name = 'quantile')

In [47]:
results = pd.Series(draws).groupby(bins_1).agg(['count', 'min', 'max']).reset_index()

In [48]:
results

,quantile,count,min,max
0,Q1,250,-3.154663,-0.665065
1,Q2,250,-0.664987,-0.044539
2,Q3,250,-0.044414,0.617270
3,Q4,250,0.618942,3.265844


#### Better performance with categorials

In [49]:
N = 10000000

In [50]:
draws = pd.Series(np.random.randn(N))

In [52]:
labels = pd.Series(['pen', 'ruler', 'book','cake'] * (N//4))

In [57]:
categories = labels.astype('category')

In [58]:
labels.memory_usage()

80000080

In [59]:
categories.memory_usage()

10000272

GroupBy operations can be significantly faster with categoricals because the underly‐ing algorithms use the integer-based codes array instead of an array of strings.

### Categorical Methods

In [60]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)

In [61]:
cat_s = s.astype('category')

In [62]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [63]:
# cat provides access to categorical methods
cat_s.cat

In [64]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [68]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [69]:
cats2 = cat_s.cat.set_categories(actual_categories)

In [70]:
cats2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [71]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [73]:
cats2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [74]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

In [75]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [76]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

#### Creating dummy variables for modeling

In [77]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [78]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 12.2 Advanced GroupBy Use

In [81]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4, 'value': np.arange(12.)})

In [82]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [83]:
g = df.groupby('key').value

In [85]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [86]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [87]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [88]:
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [90]:
g.transform(lambda x: x.rank(ascending = False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [91]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [92]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [93]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### Grouped Time Resampling

In [97]:
N = 15

In [98]:
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

In [99]:
df = pd.DataFrame({'time': times, 'value': np.arange(N)})

In [100]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [101]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [102]:
df2 = pd.DataFrame({'time': times.repeat(3),
....: 'key': np.tile(['a', 'b', 'c'], N),
....: 'value': np.arange(N * 3.)})

In [104]:
df2

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0
7,2017-05-20 00:02:00,b,7.0
8,2017-05-20 00:02:00,c,8.0
9,2017-05-20 00:03:00,a,9.0


In [105]:
time_key = pd.TimeGrouper('5min')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


In [106]:
resampled = (df2.set_index('time')
....: .groupby(['key', time_key])
....: .sum())

In [107]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [108]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


## 12.3 Techniques for Method Chaining

In [ ]:
# Confusing to me